Įvertinsime, kiek `sqlite3` yra spartus grupuojant duomenis. Uždavinys skamba taip: koks yra greičiausias būdas trimačių taškų masyvui grąžinti sutampančių taškų indeksus?

Pavyzdžiui turint masyvą

```cubes = np.array([[1233038, 8326521, 796], [1232105, 8327211, 822], [1233296, 8326274, 798], [1233296, 8326274, 798], [1233489, 8326333, 790], [1233038, 8326521, 796], [1233599, 8326360, 790], [1233489, 8326333, 790], [1233038, 8326521, 796], [1233038, 8326521, 796], [1233296, 8326274, 798], [1233489, 8326333, 790], [1233599, 8326360, 790], [1232105, 8327211, 822], [1232105, 8327211, 822], [1233678, 8326260, 821], [1233678, 8326260, 821], [1233599, 8326360, 790], [1232105, 8327211, 822], [1233296, 8326274, 798], [1233489, 8326333, 790], [1233599, 8326360, 790], [1233678, 8326260, 821], [1233678, 8326260, 821]])```

grupės atrodo taip:

```{(1232105, 8327211, 822): [1, 13, 14, 18]), 
(1233038, 8326521, 796): [0, 5, 8, 9], 
(1233296, 8326274, 798): [2, 3, 10, 19], 
(1233489, 8326333, 790): [4, 7, 11, 20], 
(1233599, 8326360, 790): [6, 12, 17, 21], 
(1233678, 8326260, 821): [15, 16, 22, 23]}```

### Python: greičiausias įmanomas būdas su `pandas` + `numexpr` (0.2 sec)

In [71]:
import numexpr as ne
import pandas as pd
import numpy as np
import time
cubes = np.load('cubes.npz')['array'][:1000000]

def to_indices_with_pandas(cubes):
    def dimensionality_reduction(cubes):
        # cubes = cubes - np.min(cubes, axis=0) #in case some coords are negative
        cubes = cubes.astype(np.int64)
        s0, s1 = cubes[:, 0].max() + 1, cubes[:, 1].max() + 1
        d = {'s0': s0, 's1': s1, 'c0': cubes[:, 0], 'c1': cubes[:, 1], 'c2': cubes[:, 2]}
        c1D = ne.evaluate('c0+c1*s0+c2*s0*s1', d)
        return c1D

    cubes = dimensionality_reduction(cubes)
    result = pd.DataFrame(cubes).groupby([0]).indices
    return result
    
t = time.time()
idx = to_indices_with_pandas(cubes)
print('TIME:', round(time.time()-t, 3))
print('RESULT:',end='\n')
i = 0
for n in idx:
    print(f'{n} -> {idx[n]}')
    i+=1
    if i>10: break

TIME: 0.147
RESULT:
8117601110684167 -> [190350]
8117601171155828 -> [278942]
8117601183497013 -> [135429]
8117601183497014 -> [132068]
8117601184731130 -> [131403 133404]
8117601189667451 -> [975861 976708]
8117601189667453 -> [960602]
8117601189667454 -> [954683 954684 955526 955527 956374 956375 957219 958063 958909 958910
 959757]
8117601189667455 -> [949596 951291]
8117601190901570 -> [953841 953842 958065]
8117601190901571 -> [946210 947060 947061 947904 948750 949597 949598 949599 950445 950446
 951292 951293 952143 952144 952992]


### Python: duomenų bazių panaudojimas su `sqlite3` (3.5 sec.)

In [72]:
import sqlite3

def to_indices_with_sqlite(cubes):
    cubes = cubes.tolist()
    conn = sqlite3.connect('pointcloud.db')
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS points''')

    
    conn.commit(); print('initially:', '\n', c.fetchall())

    c.execute('''CREATE TABLE points(id integer primary key, x integer, y integer, z integer)''')
    c.executemany('INSERT INTO points(x,y,z) VALUES (?, ?, ?)', cubes)
    
    conn.commit(); print('after Create:', '\n', c.fetchall())
    
    c.execute('SELECT GROUP_CONCAT(id) FROM points GROUP BY x, y, z')
    output = c.fetchall()
    
    conn.commit(); print('after Read:', '\n', output[:10])
    
    c.execute('UPDATE points SET z=-1 WHERE z>800')
    c.execute('SELECT x, y, z FROM points ORDER BY z LIMIT 5')
    
    conn.commit(); print('after Update:', '\n', c.fetchall())
    
    c.execute('DELETE FROM points WHERE z=-1')
    c.execute('SELECT x, y, z FROM points ORDER BY z LIMIT 5')
    conn.commit(); print('after Delete:', '\n', c.fetchall())
    
    c.execute('DELETE FROM points')
    c.execute('SELECT x, y, z FROM points ORDER BY z LIMIT 5')
    conn.commit(); print('after complete Delete:', '\n', c.fetchall())
    
    c.execute('DROP TABLE points')
    conn.commit() 
    conn.close()
    return output

t = time.time()
idx = to_indices_with_sqlite(cubes)
print('TIME:', round(time.time()-t, 3))
print('RESULT: \n', idx[:10])

initially: 
 []
after Create: 
 []
after Read: 
 [('999811',), ('997286,998127,998128,998129,998966,998967,998968,998969,999812,999813,999814,999815',), ('994758,995602,995603,995604,996446,996447,996448,996449,997287,997288,997289,997290,998130,998131,998132,998133,998970,998971,998972,998973,999816,999817,999818,999819',), ('993918,994759,995605,996450,997291,998134,998974,999820',), ('998975,999821',), ('998137,998138,998976,998977,999822,999823,999824',), ('997296,997297,998139,998140,998978,998979,998980,999825,999826',), ('996457,997298,998141,998981,999827',), ('999797',), ('997274,998115,998116,998953,998954,998955,998956,999798,999799,999800',)]
after Update: 
 [(1234089, 8326079, -1), (1234094, 8326034, -1), (1234094, 8326034, -1), (1234094, 8326034, -1), (1234094, 8326034, -1)]
after Delete: 
 [(1234115, 8326157, 789), (1234115, 8326157, 789), (1234115, 8326157, 789), (1234115, 8326156, 789), (1234115, 8326157, 789)]
after complete Delete: 
 []
TIME: 8.028
RESULT: 
 [('99981

### Python: grupavimas naudojant standartines bibliotekas (1 sec.)

In [69]:
from collections import defaultdict

def to_indices_with_python(cubes):
    res = defaultdict(list)
    for idx, elem in enumerate(cubes):
        res[elem.tobytes()].append(idx)
    return res


t = time.time()
idx = to_indices_with_python(cubes)
print('TIME:', round(time.time()-t, 3))
print('RESULT:',end='\n')
i = 0
for n in idx:
    print(f'{n} -> {idx[n]}')
    i+=1
    if i>10: break

TIME: 0.964
RESULT:
b'\xc3\xd4\x12\x00\r\x0c\x7f\x00\x15\x03\x00\x00' -> [0, 1, 2, 4, 5, 8]
b'\xc3\xd4\x12\x00\x0c\x0c\x7f\x00\x15\x03\x00\x00' -> [3, 6, 7, 11]
b'\xc2\xd4\x12\x00\r\x0c\x7f\x00\x15\x03\x00\x00' -> [9, 13, 14, 20, 27, 28, 36, 45, 46, 57]
b'\xc3\xd4\x12\x00\x0b\x0c\x7f\x00\x15\x03\x00\x00' -> [10, 15]
b'\xc2\xd4\x12\x00\x0c\x0c\x7f\x00\x15\x03\x00\x00' -> [12, 17, 18, 19, 25, 26, 33, 34, 35, 43, 44, 54, 55, 56, 66]
b'\xc2\xd4\x12\x00\x0b\x0c\x7f\x00\x15\x03\x00\x00' -> [16, 22, 23, 24, 30, 31, 32, 40, 41, 42, 52, 53, 64, 65, 77]
b'\xc2\xd4\x12\x00\n\x0c\x7f\x00\x15\x03\x00\x00' -> [21, 29, 38, 39, 50, 51, 61, 62, 63, 74, 75, 76, 89]
b'\xc2\xd4\x12\x00\t\x0c\x7f\x00\x15\x03\x00\x00' -> [37, 47, 48, 49, 58, 59, 60, 72, 73, 86, 87, 88, 103]
b'\xc1\xd4\x12\x00\x0c\x0c\x7f\x00\x15\x03\x00\x00' -> [67, 80, 81, 94, 95, 96, 111, 112, 128, 129, 130, 146, 147, 148, 165, 166]
b'\xc1\xd4\x12\x00\r\x0c\x7f\x00\x15\x03\x00\x00' -> [68, 69, 82, 97, 113, 114, 131, 149]
b'\xc2\xd4\x12\x0